In [1]:
import timeit
import matplotlib.pyplot as plt
import pickle
from mnist import MNIST
from time import time

In [2]:
print("Loading dataset....")
mndata = MNIST('../mnist_dataset')
X_train, y_train = mndata.load_training()
X_train = (mndata.process_images_to_numpy(X_train)/255)
X_test, y_test = mndata.load_testing()
X_test = (mndata.process_images_to_numpy(X_test)/255)
print("Done.")

Loading dataset....
Done.


In [3]:
def one_hot_encode(labels):
    y_inp=np.zeros((len(labels),10))
    for ind,val in enumerate(labels):
        y_inp[ind][val]=1
    return y_inp

In [4]:
def batch_gen(st=0,size=20,validate=False):
    st=st%60000
    if validate:
        X=X_test[st:st+size].reshape(-1,28,28,1)
        labels=y_test[st:st+size]
    else:
        X=X_train[st:st+size].reshape(-1,28,28,1)
        labels=y_train[st:st+size]
    y=one_hot_encode(labels)
    return X,y

In [5]:
from nnet.network import Sequential,layers
from nnet.layers import conv2d,max_pool,flatten,dense,dropout
from nnet import functions
import numpy as np

Seed: 302
Seed: 862


In [15]:
def generator():
    model=Sequential()
    model.add(dense(256,input_shape=(100),activation=functions.relu))
    model.add(dense(512,activation=functions.relu))
    model.add(dense(1024,activation=functions.relu))
    model.add(dense(784,activation=functions.tanh))
    return model
g=generator()

In [16]:
g.summary()

⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽
Layer (type)               Output Shape             Activation        Param #
input_layer(InputLayer)   (None, 100)                echo             0
__________________________________________________________________________________________
0 dense(dense)            (None, 256)                relu             25856
__________________________________________________________________________________________
1 dense(dense)            (None, 512)                relu             131584
__________________________________________________________________________________________
2 dense(dense)            (None, 1024)               relu             525312
__________________________________________________________________________________________
3 dense(dense)            (None, 784)                tanh             803600
Total Params: 1,486,352
Trainable Params: 1,486,352
Non-trainable Params: 0


In [ ]:
def discriminator():
    model=Sequential()
    model.add(dense(1024,input_shape=(784),activation=functions.relu))
    model.add(dropout(0.3))
    model.add(dense(512,activation=functions.relu))
    model.add(dropout(0.3))
    model.add(dense(256,activation=functions.relu))
    model.add(dense(1,activation=functions.sigmoid))
    return model
d=discriminator()